In [ ]:
# summary notebook

## Preprocessing steps
1. Used autocomplete in domain.com to get all the suburbs (web scraping)
2. Sent 22364 requests across a few days to scrape every property by suburb, in particular by generating and scraping the links
3. Download external datasets: crime, population and income from census, train station distances (## note: ischrone map)
4. Found the location coordinates of property data and used the geospatial suburb data to pinpoint the location of the property
5. Combined other datasets based on similar methods

## Insights on modelling

1. What are the most important internal and external features in predicting rental prices?
feature analysis PCA
Spearman correlation

Internal

The housing location (suburbs, distance to CBD)
Number of bedrooms, baths, and park (area in general)

External

Distance to train stations
Income, crime, etc need to look at the plots

2. What are the top 10 suburbs with the highest predicted growth rate?
Need to follow up with Ash on trees

3. What are the most liveable and affordable suburbs according to your chosen metrics?
Look for suburbs that has low crime rate, high income, low rent price, ideally closer to CBD and train stations that are well-spaced
Might want to standardise by dividing income with rent price

## Prediction
Made linear models for each external dataset (crime, income, and population).
Using data from previous years, predict growth rates and expected values of these statistics for each suburb in 2023 to 2025.
Using these predictions, we can then input it into the rental data model to see which suburb has the greatest growth rates in terms of rental cost. This operates under the assumption that external dataset growth will give us a good indication on the rental price growth. We found that all features (crime, income, population) increase in the future, unsurprisingly.

In [ ]:
## import images

## Modelling

1. Neural Network - Multi layer perceptron
Low accuracy on the overall dataset (R^2 = 0.31).
When suburbs were divided into groups based on distance to CBD, and NN was trained on each of these groups, R^2 increased to 0.51.
Additional feature was created for this, which is the distance between suburb and CBD. The data was also stratified for testing, and grid search was used to find optimal parameters.

2. Tree
Used an xgboost tree to model all features and tie them into rent prices by suburb. We ended up with a high R^2 (0.98).


Both tree and neural network automatically perform feature selections. Our finding on the neural network is not the ideal model, and so it was used to inform us that a different model might be better, so we did an xgboost. 

While there are a lot of property data, the number of properties per suburb ends up being quite low since there are many subrubs in Victoria. This explains why NN does not work well when stratified. When we divide these into four rings based on distance to CBD instead of suburbs, we get much better performance. However, this diminishes our ability to predict which subrub grows the most.

for each suburb, we find the average house (average number of beds, baths, etc and round it to nearest int), and from the average house we can make predictions for the price for 2022. Once we have those predictions, we can predict on the average house again with the same number of average beds, baths. And using the crime, income, population data predictions by suburb, this will give us a second value for rent for 2023. Since we have two numbers, we can calculate the growth rate for 2024 and 2025.